In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import re
from scipy.stats import spearmanr
import csv

import warnings
warnings.filterwarnings("ignore")

In [2]:
# setting = "wd=0.01_lr=3e-5"
setting = "wd=0.0_lr=5e-5"

In [3]:
def read_nodes(file_path):
    int_list = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            for item in row:
                try:
                    int_list.append(int(item))
                except ValueError:
                    print(f"Warning: '{item}' could not be converted to an integer and was skipped.")
    return int_list



def calculate_one(path):
    # score = torch.load(path, map_location=torch.device('cpu'))  # _test_0225_regroup
    score = torch.load(path, map_location=torch.device('cpu'))
    print(score)
    # score = torch.rand(5000, 500)
    print("score shape:", score.shape)

    nodes_str = [f"./checkpoints/{setting}/{i}/train_index.csv" for i in range(50)]
    # for i in range(50):
    #     nodes_str.append(f"./checkpoints/{i}/train_index.csv")

    full_nodes = [i for i in range(4656)]

    node_list = []
    for node_str in nodes_str:
        numbers = read_nodes(node_str)
        index = []
        for number in numbers:
            index.append(full_nodes.index(number))
        node_list.append(index)

    loss_list = torch.load(f"./results/{setting}/gt.pt", map_location=torch.device('cpu')).detach()

    approx_output = []
    for i in range(len(nodes_str)):
        score_approx_0 = score[node_list[i], :]
        sum_0 = torch.sum(score_approx_0, axis=0)
        approx_output.append(sum_0)

    print(len(loss_list), loss_list[0].shape)
    print(len(approx_output), approx_output[0].shape)

    res = 0
    counter = 0
    for i in range(481):
        tmp = spearmanr(np.array([approx_output[k][i] for k in range(len(approx_output))]),
                        np.array([loss_list[k][i].numpy() for k in range(len(loss_list))])).statistic
        if np.isnan(tmp):
            print("Numerical issue")
            continue
        res += tmp
        counter += 1

    print(counter)

    return res/counter, loss_list, approx_output

# path = f"results/{setting}/IF/Linear_LayerNorm/default_SJLT_R-64(U)_thrd-0.0_randrop-0.2.pt"
# print(torch.load(f"results/{setting}/IF/Linear_LayerNorm/default_SJLT_R-64(U)_thrd-0.0_randrop-0.2_profile.pt"))
path = "./LoGra/gpt_influence.pt"
print(calculate_one(path)[0])

tensor([[-1.5449e+02, -1.4994e+02, -3.6882e+02,  ...,  2.3526e+02,
         -2.5612e+02, -1.1065e+04],
        [-5.2611e+01,  3.2672e+02, -4.8657e+02,  ...,  7.9459e+02,
          2.9331e+02, -3.3699e+01],
        [-9.9731e+01,  3.7158e+02, -1.7222e+02,  ..., -1.9625e+02,
         -1.4427e+01,  7.4074e+03],
        ...,
        [ 3.9775e+02, -1.9990e+02,  3.0909e+02,  ..., -2.1358e+02,
         -1.8581e+02, -1.1298e+03],
        [ 2.2006e+01, -1.1110e+01,  1.4881e+02,  ..., -1.3802e+02,
         -1.1545e+02, -1.1734e+04],
        [-5.3875e+02, -5.4027e+01,  6.5142e+02,  ...,  2.0387e+02,
          3.8810e+01, -1.4787e+04]])
score shape: torch.Size([4656, 481])
50 torch.Size([481])
50 torch.Size([481])
481
0.2500514329075106


In [4]:
print(torch.load(f"./results/{setting}/gt.pt", map_location=torch.device('cpu')))
print(torch.load(f"./results/{setting}/IF/Linear/default_SJLT_R-16(U)_thrd-0.0_randrop-0.0.pt", map_location=torch.device('cpu'), weights_only=False))

tensor([[-2.6565, -2.4843, -2.9398,  ..., -2.9156, -2.9455, -3.2442],
        [-2.6787, -2.5410, -2.9881,  ..., -2.9291, -2.9266, -3.2387],
        [-2.6541, -2.4947, -2.9545,  ..., -2.9190, -2.9386, -3.2585],
        ...,
        [-2.6674, -2.5061, -2.9756,  ..., -2.9111, -2.9182, -3.2272],
        [-2.6664, -2.5151, -2.9562,  ..., -2.9259, -2.9536, -3.2526],
        [-2.6667, -2.5240, -2.9689,  ..., -2.9531, -2.9314, -3.2552]])
{'score': tensor([[ -59.0004,   -4.4000,  -22.7205,  ...,  203.9437,   93.7282,
         -101.4411],
        [ -42.5184,   15.9472,   19.0930,  ...,   47.7053,   47.6861,
          -28.1987],
        [ -52.5495,   62.8531,  -10.5450,  ...,  -44.1897,  101.4645,
          112.9300],
        ...,
        [ 135.3539,  -25.3235,  -66.0985,  ...,  -35.5701,   56.8760,
           69.9570],
        [  50.5053,   78.3168,   63.3604,  ...,   34.7661,   18.9247,
          -23.6036],
        [ -68.3581,    9.1142,  112.6794,  ...,  -39.9870,   56.2187,
         -153.6979